In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import numpy as np

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()


In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos={s:i for i,s in stoi.items()}




In [4]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
C = torch.randn(27, 2)

In [7]:
C.shape

torch.Size([27, 2])

In [8]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [9]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)


In [10]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [11]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
C.shape

torch.Size([27, 2])

In [12]:
logits = h @ W2 + b2
counts = logits.exp()

prob = counts / counts.sum(1, keepdims=True)

prob.shape

loss = -prob[torch.arange(32), Y].log().mean()
loss



tensor(17.9217)

In [13]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [14]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [15]:
for p in parameters:
  p.requires_grad = True

In [16]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [17]:
lri = []
lossi = []
stepi = []

In [18]:
for i in range(200):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))
  
  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

#print(loss.item())

In [19]:
C[Xtr[ix]].shape

torch.Size([32, 3, 10])

In [20]:
C.shape

torch.Size([27, 10])

In [21]:
C[Xtr].shape

torch.Size([182625, 3, 10])

In [22]:
Xtr[ix].shape

torch.Size([32, 3])

In [23]:
ix.shape

torch.Size([32])

In [24]:
Xtr.shape

torch.Size([182625, 3])

In [26]:
C[torch.tensor([context])]

tensor([[[-0.4912,  0.5113, -0.0660, -1.4757, -0.5668,  1.5861,  0.0654,
          -0.1858, -0.7234, -0.2881],
         [-0.4661,  0.9640, -0.5451, -1.4131,  1.1127,  2.9376,  1.8981,
          -1.6710, -0.1221, -0.7479],
         [ 2.0666,  0.5714,  0.0353, -1.1745,  0.8873,  0.0731, -2.0764,
           0.9232,  0.8991,  1.2508]]], grad_fn=<IndexBackward0>)

In [28]:
X[-1]

tensor([8, 9, 1])

In [ ]:
C[torch.tensor([context])].shape